In [ ]:
import nltk
from nltk.corpus import brown
#from nltk.tokenize import word_tokenize
import numpy as np
#import pandas as pd
import re
import copy
#import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import random #for shuffling list
from sklearn.model_selection import train_test_split
import tensorflow as tf
import gensim

nltk.download('punkt')
nltk.download('brown')
nltk.download('universal_tagset')
dataset = brown.tagged_sents(tagset='universal')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


In [ ]:
# tags_and_words=brown.tagged_words(tagset='universal')
# print(len(tags_and_words))

In [ ]:
#train_tagged_sents,val_tagged_sents=train_test_split(dataset,test_size=0.2,random_state=40)
#val_sents,test_sents=train_test_split(remaining,test_size=0.1,train_size=0.2,random_state=40)

In [ ]:
import gensim.downloader as api
import json
info = api.info()
#print(json.dumps(info, indent=4))
print(api.load('word2vec-google-news-300', return_path=True))

[==================================================] 100.0% 1662.8/1662.8MB downloaded
/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [ ]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz', binary=True)

In [ ]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# a=[]
# for sent in dataset:
#     a.append(len(sent))
# print(max(a))
# print(min(a))

180
1


In [ ]:
def get_feature_dataset_train(train_tagged_sents):  
    #words_and_tags = brown.tagged_sents(tagset='universal')

    words_in_corpus = []
    tags_in_corpus = []

    for tagged_sent in train_tagged_sents:
        for item in tagged_sent:
            words_in_corpus.append(item[0])
            tags_in_corpus.append(item[1])

    words_in_corpus = list(set(words_in_corpus))
    tags_in_corpus = list(set(tags_in_corpus))

    words_in_corpus.sort()
    tags_in_corpus.sort()

    print(len(words_in_corpus))
    print(tags_in_corpus)

    word_dictionary = dict()
    tag_dictionary = dict()

    for i in range ( len(tags_in_corpus) ):
        tag_dictionary[tags_in_corpus[i]] = i # 0 based indexing of IDs

    for i in range ( len(words_in_corpus) ):
        word_dictionary[words_in_corpus[i]] = i # 0 based indexing of IDs

    index_for_word_not_in_corpus = len(words_in_corpus)
    
    def suffix_features(word): #input is the string form of word

        features = []
        flag = False

        # ---- x ---- Noun Suffix ---- x ----
        noun_suffixes = ['acy', 'al', 'ance', 'ence', 'dom', 'er', 'or', 'ism',
                         'ist', 'ity', 'ty', 'ment', 'ness', 'ship', 'sion',
                         'tion', 'ct']

        for i in noun_suffixes:
            if len(re.findall(i+"$", word)) > 0:
                flag = True
                break

        if flag :
            features.append(1)
        else: 
            features.append(0)

        flag = False

        # ---- x ---- Verb Suffix ---- x ----
        verb_suffixes = ['es', 'ed', 'ate', 'en', 'ify', 'fy', 'ize', 'ise',
                         'ers', 'ingly']

        for i in verb_suffixes:
            if len(re.findall(i+"$", word)) > 0:
                flag = True
                break

        if flag :
            features.append(1)
        else: 
            features.append(0)

        flag = False

        # ---- x ---- Adjective Suffix ---- x ----
        adj_suffixes = ['able', 'ible', 'al', 'esque', 'ful', 'ic', 'ical',
                        'ious', 'ous', 'ish', 'ive', 'less', 'y']

        for i in adj_suffixes:
            if len(re.findall(i+"$", word)) > 0:
                flag = True
                break

        if flag :
            features.append(1)
        else: 
            features.append(0)


        return features

    def feature_matrix(sentence, tag_list):

        assert len(sentence) == len(tag_list)

        n = len(sentence)
        f_mat = []
        y_arr=[]
        for i in range (n):
            f_word = []

            # ------ x ------ 1st feature - ID of word in dictionary ------ x ------

            f_word.append(word_dictionary[sentence[i]])

            # ------ x ------ 2nd feature - ID of previous word in dictionary ------ x ------

            if i != 0 : #valid if it is not the first word
                f_word.append(word_dictionary[sentence[i - 1]])
            else:
                f_word.append(-1)

            # ------ x ------ 3rd feature - ID of next word in dictionary ------ x ------

            if i != n - 1 : #valid if it is not the last word
                f_word.append(word_dictionary[sentence[i + 1]])
            else:
                f_word.append(-1)        

            # ------ x ------ 4th feature - POS tag of previous word ------ x ------

            if i!=0: #valid if it is not the first word
                f_word.append(tag_dictionary[tag_list[i-1]])
            else:
                f_word.append(-1)

            # ------ x ------ 5th, 6th and 7th feature - Noun, Verb and Adjective suffix check of current word ------ x ------

            f_word = f_word + suffix_features(sentence[i])
            #print(str(len(f_word)) + "    " + str(len(suffix_features(sentence[i]))))

            # ------ x ------ 8th, 9th and 10th feature - Noun, Verb and Adjective suffix check of previous word ------ x ------

            if i != 0 : #valid if it is not the first word
                f_word = f_word + suffix_features(sentence[i - 1])
            else:
                f_word = f_word + [-1, -1, -1]

            # ------ x ------ 11th, 12th and 13th feature - Noun, Verb and Adjective suffix check of next word ------ x ------

            if i != n - 1 : #valid if it is not the last word
                f_word = f_word + suffix_features(sentence[i + 1])
            else:
                f_word = f_word + [-1, -1, -1]
                
            # ------ x ------ pos tag of current word as y ------ x ------
            y_arr.append(tag_dictionary[tag_list[i]])
            
            f_mat.append(f_word)
            #print(len(f_word))

        return f_mat,y_arr
    
    train_mat=[]
    train_y=[]
    for tagged_sent in train_tagged_sents:
        sentence=[]
        tag_list=[]
        for item in tagged_sent:
            sentence.append(item[0])
            tag_list.append(item[1])
        f_mat,y_arr=feature_matrix(sentence,tag_list)
        train_mat+=f_mat
        train_y+=y_arr
    return train_mat,train_y

In [ ]:
# train_mat,train_y=get_feature_dataset_train(train_tagged_sents)

50647
['.', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB', 'X']


In [ ]:
import string
#global failed
#failed=0
def get_feature_dataset_train_w2v(train_tagged_sents):
    # failed=0
    # total=0  
    #words_and_tags = brown.tagged_sents(tagset='universal')

    words_in_corpus = []
    tags_in_corpus = []

    for tagged_sent in train_tagged_sents:
        for item in tagged_sent:
            words_in_corpus.append(item[0])
            tags_in_corpus.append(item[1])

    words_in_corpus = list(set(words_in_corpus))
    tags_in_corpus = list(set(tags_in_corpus))

    words_in_corpus.sort()
    tags_in_corpus.sort()

    print(len(words_in_corpus))
    print(tags_in_corpus)

    word_dictionary = dict()
    tag_dictionary = dict()

    for i in range ( len(tags_in_corpus) ):
        tag_dictionary[tags_in_corpus[i]] = i # 0 based indexing of IDs

    for i in range ( len(words_in_corpus) ):
        word_dictionary[words_in_corpus[i]] = i # 0 based indexing of IDs

    index_for_word_not_in_corpus = len(words_in_corpus)
    
    

    def feature_matrix(sentence, tag_list):
        # failed=0
        # total=0
        assert len(sentence) == len(tag_list)

        n = len(sentence)
        f_mat = []
        y_arr=[]
        for i in range(n):
            # total+=1
            try:
                f_mat.append(w2v_model[sentence[i]])
                #f_mat.append(w2v_model.wv[sentence[i]])
                y_arr.append(tag_dictionary[tag_list[i]])
            except:
                # failed+=1
                #pass
                #f_mat.append(np.random.normal(loc=0.0,scale=0.1,size=300))
                #f_mat.append(w2v_model.wv[sentence[i]])
                #y_arr.append(tag_dictionary[tag_list[i]])
                
                f_mat.append(np.random.normal(loc=0.0,scale=0.1,size=300))
                y_arr.append(tag_dictionary[tag_list[i]])

            # ------ x ------ pos tag of current word as y ------ x ------
            #y_arr.append(tag_dictionary[tag_list[i]])
            
            #f_mat.append(w2v_model.wv[sentence[i]])
            #print(len(f_word))

        return f_mat,y_arr
    
    train_mat=[]
    train_y=[]
    
    for tagged_sent in train_tagged_sents:
        sentence=[]
        tag_list=[]
        for item in tagged_sent:
            sentence.append(item[0])
            tag_list.append(item[1])
        f_mat,y_arr=feature_matrix(sentence,tag_list)
        
        train_mat+=f_mat
        train_y+=y_arr
    return train_mat,train_y

In [ ]:
# train_mat_w2v,train_y=get_feature_dataset_train_w2v(train_tagged_sents)

50647
['.', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB', 'X']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [ ]:

# #tf.debugging.set_log_device_placement(True)

# model=tf.keras.models.Sequential([
#     #tf.keras.layers.LayerNormalization(),
#     tf.keras.layers.Dense(64,activation='relu'),
#     tf.keras.layers.LayerNormalization(),
#     tf.keras.layers.Dense(12,activation=tf.keras.activations.softmax)
# ])

In [ ]:
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

In [ ]:
# model.compile(optimizer='adam',
#               loss=loss_fn,
#               metrics=['accuracy'])

In [ ]:
# BATCH_SIZE = 64
# # SHUFFLE_BUFFER_SIZE = 100
# train_tf_data = tf.data.Dataset.from_tensor_slices((train_mat, train_y)).batch(BATCH_SIZE)
# model.fit(train_tf_data, epochs=10)

KeyboardInterrupt: ignored

In [ ]:
# #w2v model
# BATCH_SIZE = 64
# # SHUFFLE_BUFFER_SIZE = 100
# train_tf_data = tf.data.Dataset.from_tensor_slices((train_mat_w2v, train_y)).batch(BATCH_SIZE)
# model.fit(train_tf_data, epochs=10)

Epoch 1/10
11014/11014 [==============================] - 30s 3ms/step - loss: 0.1845 - accuracy: 0.9382
Epoch 2/10
11014/11014 [==============================] - 30s 3ms/step - loss: 0.1492 - accuracy: 0.9466
Epoch 3/10
11014/11014 [==============================] - 29s 3ms/step - loss: 0.1422 - accuracy: 0.9481
Epoch 4/10
11014/11014 [==============================] - 29s 3ms/step - loss: 0.1383 - accuracy: 0.9490
Epoch 5/10
11014/11014 [==============================] - 29s 3ms/step - loss: 0.1358 - accuracy: 0.9496
Epoch 6/10
11014/11014 [==============================] - 29s 3ms/step - loss: 0.1338 - accuracy: 0.9501
Epoch 7/10
11014/11014 [==============================] - 29s 3ms/step - loss: 0.1323 - accuracy: 0.9504
Epoch 8/10
11014/11014 [==============================] - 29s 3ms/step - loss: 0.1310 - accuracy: 0.9507
Epoch 9/10
11014/11014 [==============================] - 29s 3ms/step - loss: 0.1300 - accuracy: 0.9510
Epoch 10/10
11014/11014 [==============================

In [ ]:
def sentence_decoding(training_sentences, test_sentences):
    train_mat_w2v,train_y=get_feature_dataset_train_w2v(training_sentences)
    train_mat_w2v=np.array(train_mat_w2v)
    train_y=np.array(train_y)
    print("training sets made ---------------------------------------")
    
    #tf.debugging.set_log_device_placement(True)

    model=tf.keras.models.Sequential([
        #tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Dense(12,activation=tf.keras.activations.softmax)
    ])
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
    BATCH_SIZE = 4
    # SHUFFLE_BUFFER_SIZE = 100
    print("starting making tf data")
    train_tf_data = tf.data.Dataset.from_tensor_slices((train_mat_w2v, train_y)).batch(BATCH_SIZE)
    print("training started")
    model.fit(train_tf_data, epochs=5)
    
    test_mat_w2v,test_y=get_feature_dataset_train_w2v(test_sentences)
    test_tf_data = tf.data.Dataset.from_tensor_slices(test_mat_w2v).batch(BATCH_SIZE)
    one_hot_predictions=model.predict(test_tf_data)
    predictions=[]
    for i in range(len(one_hot_predictions)):
        arr=one_hot_predictions[i].tolist()
        
        predictions.append(arr.index(max(arr)))

    #print("test_y",test_y)
    #print("predictions",predictions)
    print(confusion_matrix(test_y,predictions))
    return test_y, predictions

In [ ]:
def accuracy_per_fold(training_sentences, test_sentences):
    test_y,predictions = sentence_decoding(training_sentences, test_sentences)
    
    f1 = 0
    prec = 0
    recall = 0
    

    #for i in range( len(actual_tags) ):
    #    f1 += f1_score(actual_tags[i], predicted_tags[i], average='weighted')
    #    prec += precision_score(actual_tags[i], predicted_tags[i], average='weighted')
    #    recall += recall_score(actual_tags[i], predicted_tags[i], average='weighted')
    per_pos=[[] for i in range(12)]
    #per_pos_act=[[] for i in range(0,12)]
    for i in range(len(test_y)):
        per_pos[test_y[i]].append(predictions[i])
    print("my_fold------------------------")
    for i in range(len(per_pos)):
        print(i,"-----")
        true_tag=[i for j in range(len(per_pos[i]))]
        print("precision",precision_score(per_pos[i],true_tag, average='weighted'))
        print("recall",recall_score(per_pos[i],true_tag, average='weighted'))
        print("f1",f1_score(per_pos[i],true_tag, average='weighted'))
    print("overall for the batch:")
    precision=precision_score(test_y,predictions, average='weighted')
    recall=recall_score(test_y,predictions, average='weighted')
    f1=f1_score(test_y,predictions, average='weighted')
    print("precision",precision)
    print("recall",recall)
    print("f1",f1)

        
    return [precision, recall, f1]
    #return [f1/len(actual_tags), prec/len(actual_tags), recall/len(actual_tags)]

#total_acc_for_split1 = accuracy_per_fold(all_sentences[:57000], all_sentences[56000:56010])
#total_acc_for_split2 = accuracy_per_fold(all_sentences[:57000], all_sentences[57000:57010])

#print("\nTotal accuracy for the above split1: " + str(total_acc_for_split1))
#print("Total accuracy for the above split2: " + str(total_acc_for_split2) + "\n")
#exit(0)

def cross_validation_accuracy(all_sentences = dataset):
    all_accuracies = []
    total_acc = [0, 0, 0]

    num_sentences = len(dataset)

    fold_size = int(num_sentences / 5)

    print(fold_size)

    fold1_train=dataset[:4*fold_size]
    fold1_test=dataset[4*fold_size:]
    fold2_train=dataset[:3*fold_size]+dataset[4*fold_size:]
    fold2_test=dataset[3*fold_size:4*fold_size]
    fold3_train=dataset[:2*fold_size]+dataset[3*fold_size:]
    fold3_test=dataset[2*fold_size:3*fold_size]
    fold4_train=dataset[:1*fold_size]+dataset[2*fold_size:]
    fold4_test=dataset[1*fold_size:2*fold_size]
    fold5_train=dataset[1*fold_size:]
    fold5_test=dataset[:1*fold_size]

    all_accuracies.append(accuracy_per_fold(fold1_train, fold1_test))
    print("Accuracy for fold ", "1", " is : ", str(all_accuracies[0]))
    total_acc[0] += all_accuracies[0][0]
    total_acc[1] += all_accuracies[0][1]
    total_acc[2] += all_accuracies[0][2]


    all_accuracies.append(accuracy_per_fold(fold2_train, fold2_test))
    print("Accuracy for fold ", "2", " is : ", str(all_accuracies[1]))
    total_acc[0] += all_accuracies[1][0]
    total_acc[1] += all_accuracies[1][1]
    total_acc[2] += all_accuracies[1][2]

    all_accuracies.append(accuracy_per_fold(fold3_train, fold3_test))
    print("Accuracy for fold ", "3", " is : ", str(all_accuracies[2]))
    total_acc[0] += all_accuracies[2][0]
    total_acc[1] += all_accuracies[2][1]
    total_acc[2] += all_accuracies[2][2]

    all_accuracies.append(accuracy_per_fold(fold4_train, fold4_test))
    print("Accuracy for fold ", "4", " is : ", str(all_accuracies[3]))
    total_acc[0] += all_accuracies[3][0]
    total_acc[1] += all_accuracies[3][1]
    total_acc[2] += all_accuracies[3][2]

    all_accuracies.append(accuracy_per_fold(fold5_train, fold5_test))
    print("Accuracy for fold ", "5", " is : ", str(all_accuracies[4]))
    total_acc[0] += all_accuracies[4][0]
    total_acc[1] += all_accuracies[4][1]
    total_acc[2] += all_accuracies[4][2]

    print("Cross validation accuracy is : ", str(np.array(total_acc) / 5))

In [ ]:
cross_validation_accuracy()

NameError: ignored

Model for showing demo for word2vec

In [ ]:
def sentence_decoding_model_save(training_sentences):
    train_mat_w2v,train_y=get_feature_dataset_train_w2v(training_sentences)
    
    #tf.debugging.set_log_device_placement(True)

    model=tf.keras.models.Sequential([
        #tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Dense(12,activation=tf.keras.activations.softmax)
    ])
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
    BATCH_SIZE = 64
    # SHUFFLE_BUFFER_SIZE = 100
    train_tf_data = tf.data.Dataset.from_tensor_slices((train_mat_w2v, train_y)).batch(BATCH_SIZE)
    model.fit(train_tf_data, epochs=5)
    model.save("demo_model.h5")
    #test_mat_w2v,test_y=get_feature_dataset_train_w2v(test_sentences)
    #test_tf_data = tf.data.Dataset.from_tensor_slices(test_mat_w2v).batch(BATCH_SIZE)
    #one_hot_predictions=model.predict(test_tf_data)
    #predictions=[]
    #for i in range(len(one_hot_predictions)):
    #    arr=one_hot_predictions[i].tolist()
        
    #    predictions.append(arr.index(max(arr)))

    #print("test_y",test_y)
    #print("predictions",predictions)
    #print(confusion_matrix(test_y,predictions))
    return

In [ ]:
sentence_decoding_model_save(dataset[:int(4/5*len(dataset))])

Cross validation for FFNN model with engineered features

In [ ]:
def sentence_decoding(training_sentences, test_sentences):
    train_mat,train_y=get_feature_dataset_train(training_sentences)
    
    #tf.debugging.set_log_device_placement(True)

    model=tf.keras.models.Sequential([
        #tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Dense(12,activation='relu'),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Dense(12,activation=tf.keras.activations.softmax)
    ])
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
    BATCH_SIZE = 64
    # SHUFFLE_BUFFER_SIZE = 100
    train_tf_data = tf.data.Dataset.from_tensor_slices((train_mat, train_y)).batch(BATCH_SIZE)
    model.fit(train_tf_data, epochs=5)
    
    test_mat,test_y=get_feature_dataset_train(test_sentences)
    test_tf_data = tf.data.Dataset.from_tensor_slices(test_mat).batch(BATCH_SIZE)
    one_hot_predictions=model.predict(test_tf_data)
    predictions=[]
    for i in range(len(one_hot_predictions)):
        arr=one_hot_predictions[i].tolist()
        predictions.append(arr.index(max(arr)))

    #print("test_y",test_y)
    #print("predictions",predictions)
    print(confusion_matrix(test_y,predictions))
    return test_y, predictions

In [ ]:
def accuracy_per_fold(training_sentences, test_sentences):
    test_y,predictions = sentence_decoding(training_sentences, test_sentences)

    f1 = 0
    prec = 0
    recall = 0
    

    #for i in range( len(actual_tags) ):
    #    f1 += f1_score(actual_tags[i], predicted_tags[i], average='weighted')
    #    prec += precision_score(actual_tags[i], predicted_tags[i], average='weighted')
    #    recall += recall_score(actual_tags[i], predicted_tags[i], average='weighted')
    per_pos=[[] for i in range(12)]
    #per_pos_act=[[] for i in range(0,12)]
    for i in range(len(test_y)):
        per_pos[test_y[i]].append(predictions[i])
    print("my_fold------------------------")
    for i in range(len(per_pos)):
        print(i,"-----")
        true_tag=[i for j in range(len(per_pos[i]))]
        print("precision",precision_score(per_pos[i],true_tag, average='weighted'))
        print("recall",recall_score(per_pos[i],true_tag, average='weighted'))
        print("f1",f1_score(per_pos[i],true_tag, average='weighted'))
    print("overall for the batch:")
    precision=precision_score(test_y,predictions, average='weighted')
    recall=recall_score(test_y,predictions, average='weighted')
    f1=f1_score(test_y,predictions, average='weighted')
    print("precision",precision)
    print("recall",recall)
    print("f1",f1)

        
    return [precision, recall, f1]
    #return [f1/len(actual_tags), prec/len(actual_tags), recall/len(actual_tags)]

#total_acc_for_split1 = accuracy_per_fold(all_sentences[:57000], all_sentences[56000:56010])
#total_acc_for_split2 = accuracy_per_fold(all_sentences[:57000], all_sentences[57000:57010])

#print("\nTotal accuracy for the above split1: " + str(total_acc_for_split1))
#print("Total accuracy for the above split2: " + str(total_acc_for_split2) + "\n")
#exit(0)

def cross_validation_accuracy(all_sentences = dataset):
    all_accuracies = []
    total_acc = [0, 0, 0]

    num_sentences = len(dataset)

    fold_size = int(num_sentences / 5)

    print(fold_size)

    fold1_train=dataset[:4*fold_size]
    fold1_test=dataset[4*fold_size:]
    fold2_train=dataset[:3*fold_size]+dataset[4*fold_size:]
    fold2_test=dataset[3*fold_size:4*fold_size]
    fold3_train=dataset[:2*fold_size]+dataset[3*fold_size:]
    fold3_test=dataset[2*fold_size:3*fold_size]
    fold4_train=dataset[:1*fold_size]+dataset[2*fold_size:]
    fold4_test=dataset[1*fold_size:2*fold_size]
    fold5_train=dataset[1*fold_size:]
    fold5_test=dataset[:1*fold_size]

    all_accuracies.append(accuracy_per_fold(fold1_train, fold1_test))
    print("Accuracy for fold ", "1", " is : ", str(all_accuracies[0]))
    total_acc[0] += all_accuracies[0][0]
    total_acc[1] += all_accuracies[0][1]
    total_acc[2] += all_accuracies[0][2]


    all_accuracies.append(accuracy_per_fold(fold2_train, fold2_test))
    print("Accuracy for fold ", "2", " is : ", str(all_accuracies[1]))
    total_acc[0] += all_accuracies[1][0]
    total_acc[1] += all_accuracies[1][1]
    total_acc[2] += all_accuracies[1][2]

    all_accuracies.append(accuracy_per_fold(fold3_train, fold3_test))
    print("Accuracy for fold ", "3", " is : ", str(all_accuracies[2]))
    total_acc[0] += all_accuracies[2][0]
    total_acc[1] += all_accuracies[2][1]
    total_acc[2] += all_accuracies[2][2]

    all_accuracies.append(accuracy_per_fold(fold4_train, fold4_test))
    print("Accuracy for fold ", "4", " is : ", str(all_accuracies[3]))
    total_acc[0] += all_accuracies[3][0]
    total_acc[1] += all_accuracies[3][1]
    total_acc[2] += all_accuracies[3][2]

    all_accuracies.append(accuracy_per_fold(fold5_train, fold5_test))
    print("Accuracy for fold ", "5", " is : ", str(all_accuracies[4]))
    total_acc[0] += all_accuracies[4][0]
    total_acc[1] += all_accuracies[4][1]
    total_acc[2] += all_accuracies[4][2]

    print("Cross validation accuracy is : ", str(np.array(total_acc) / 5))

In [ ]:
cross_validation_accuracy()

11468
51461
['.', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB', 'X']
Epoch 1/5
15307/15307 [==============================] - 45s 3ms/step - loss: 1.7465 - accuracy: 0.3613
Epoch 2/5
15307/15307 [==============================] - 40s 3ms/step - loss: 1.6939 - accuracy: 0.3735
Epoch 3/5
15307/15307 [==============================] - 40s 3ms/step - loss: 1.6842 - accuracy: 0.3788
Epoch 4/5
15307/15307 [==============================] - 40s 3ms/step - loss: 1.6771 - accuracy: 0.3841
Epoch 5/5
15307/15307 [==============================] - 40s 3ms/step - loss: 1.6732 - accuracy: 0.3859
16747
['.', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB', 'X']
[[28556     0     0   146     0    33   173     0    69     0   106     0]
 [  486     0     4   119     0  1345  4145     0   103     0  3653     0]
 [  842     0   111    68     0  3300  5606     0   519     0  7988     0]
 [  939     0    12   395     0  1285  4171     0   503     0  2994

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision 

recall 0.0
f1 0.0
2 -----
precision 3.6258246006805e-05
recall 0.0060214820440490395
f1 7.208244884221553e-05
3 -----
precision 0.001470970878773983
recall 0.03835323817846393
f1 0.002833276431736161
4 -----
precision 0.0
recall 0.0
f1 0.0
5 -----
precision 0.07046466994465865
recall 0.2654518222666001
f1 0.11136681571716675
6 -----


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Re

precision 0.3147266050793197
recall 0.5610049955921246
f1 0.4032358717211367
7 -----
precision 0.0
recall 0.0
f1 0.0
8 -----
precision 0.013881024505773346
recall 0.11781775972141613
f1 0.02483593481147194
9 -----
precision 0.0
recall 0.0
f1 0.0
10 -----
precision 0.19548387072457873
recall 0.4421355795732557


...

precision 0.4577244709205921
recall 0.6765533762539302
f1 0.5460302993076497
7 -----
precision 0.0
recall 0.0
f1 0.0
8 -----
precision 0.024342693780560597
recall 0.1560214529497806
f1 0.042114605604327116
9 -----
precision 0.0
recall 0.0
f1 0.0
10 -----


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision 

precision 0.06089441519733546
recall 0.24676793794440854
f1 0.09768363998473412
11 -----
precision 0.0
recall 0.0
f1 0.0
overall for the batch:


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision 0.30006113800470013
recall 0.3783272088193002
f1 0.32317782433718273
Accuracy for fold  5  is :  [0.30006113800470013, 0.3783272088193002, 0.32317782433718273]
Cross validation accuracy is :  [0.32154204 0.36894886 0.31436652]
